## Quick and dirty trust simulation

In [ ]:
import mate
import avstack

from avstack.geometry import GlobalOrigin3D, Position

import numpy as np
import matplotlib.pyplot as plt

TINY_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=TINY_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
agents = [
    np.array([0, 0]),
    np.array([1, 0]),
    np.array([1/2, np.sqrt(3/4)])
]
radius = 0.8

objects = [
    np.array([0, 0.5]),   # in agents 0, 1
    np.array([0.2, 0.5]), # in agents 0, 1
    np.array([-0.5, 0]),  # in agent  1
    np.array([0.5, 0.5]), # in agents 0, 1, 2
    np.array([1, 0.6]),   # in agents 1, 2
]

false_positives = [
    (0, np.array([0.5, 0.3])),  # (agent 0, location)
    (0, np.array([0.45, -0.5])),
    (1, np.array([0.8, 0.4])),
]

false_negatives = [
    (0, 1),  # (agent 0, object 1)
]

detections = {}
labels = {}
for i, agent in enumerate(agents):
    dets = []
    for j, obj in enumerate(objects):
        if (i,j) in false_negatives:
            continue
        else:
            if np.linalg.norm(agent - obj) <= radius:
                dets.append(Position(x=obj, reference=GlobalOrigin3D))
    for j, fp in enumerate(false_positives):
        if fp[0] == i:
            dets.append(Position(x=fp[1], reference=GlobalOrigin3D))
    detections[i] = dets

# -- plot full picture
fig, ax = plt.subplots()
agent_colors = "rgb"
# plot agents
for agent, color in zip(agents, agent_colors):
    ax.plot(*agent, '*', markersize=7, color=color)
    circle = plt.Circle(agent, radius, color=color, alpha=0.4)
    ax.add_patch(circle)

# plot objects
for i, object in enumerate(objects):
    ax.plot(*object, 'o', markersize=5, color="black", label="Truth" if i==0 else "")

# plot false positives
for i, fp in enumerate(false_positives):
    ax.plot(*fp[1], 'x', markersize=5, color=agent_colors[fp[0]], label="FP" if i == 0 else "")

# plot false negatives
for i, fn in enumerate(false_negatives):
    ax.plot(*objects[fn[1]], '+', markersize=8, color=agent_colors[fn[0]], label="FN" if i == 0 else "")

# set limits
ax.set_xlim([-1, 2])
ax.set_ylim([-1, 1.8])
ax.set_aspect('equal', adjustable='box')
plt.legend()
plt.axis('off')
plt.tight_layout()
plt.savefig("figures/experiment_truth.pdf")
plt.show()


# -- plot detections only
fig, ax = plt.subplots()
agent_colors = "rgb"
# plot agents
for agent, color in zip(agents, agent_colors):
    ax.plot(*agent, '*', markersize=7, color=color)
    circle = plt.Circle(agent, radius, color=color, alpha=0.4)
    ax.add_patch(circle)

# plot detections
det_markers = "123"
for i_agent, dets in detections.items():
    for j, det in enumerate(dets):
        ax.plot(*det.x, det_markers[i_agent], markersize=10, alpha=1, color=agent_colors[i_agent],
                label="Detection" if j==0 else "")

# set limits
ax.set_xlim([-1, 2])
ax.set_ylim([-1, 1.8])
ax.set_aspect('equal', adjustable='box')
plt.legend()
plt.axis('off')
plt.tight_layout()
plt.savefig("figures/experiment_detections.pdf")
plt.show()



In [ ]:
class PSM:
    def __init__(self, result, dist=None, p_exist=None):
        if result:
            assert dist is not None
        self.result = result
        self.dist = dist
        self.p_exist = p_exist

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        return f"Pseudomeasurement: ({self.value}, {self.confidence})"

    @property
    def value(self):
        return self.result
    
    @property
    def confidence(self):
        return self.p_exist


# let's say we already have some tracks on objects
idx_tracks = [0,1,2,4]
tracks = [Position(obj, GlobalOrigin3D) for i, obj in enumerate(objects) if i in idx_tracks]

# cluster the detections
clusterer = avstack.modules.clustering.SampledAssignmentClusterer(assign_radius=0.05)
clusts = clusterer(detections, frame=0, timestamp=0)

# get labels by assigning to truth
truths = [Position(obj, GlobalOrigin3D) for i, obj in enumerate(objects)]
A_tru = avstack.modules.assignment.build_A_from_distance(clusts, truths)
assign_truth = avstack.modules.assignment.gnn_single_frame_assign(A_tru)

# assign clusters to existing tracks
p_exist_fix = 0.94
A = avstack.modules.assignment.build_A_from_distance(clusts, tracks)
assign = avstack.modules.assignment.gnn_single_frame_assign(A)

# plot clusters
fig, ax = plt.subplots()
agent_colors = "rgb"
# plot agents
for agent, color in zip(agents, agent_colors):
    ax.plot(*agent, '*', markersize=7, color=color)
    circle = plt.Circle(agent, radius, color=color, alpha=0.4)
    ax.add_patch(circle)

# plot objects
for i, object in enumerate(objects):
    ax.plot(*object, 'o', markersize=7, color="black", label="Truth" if i==0 else "")

# plot clusters
for j, clust in enumerate(clusts):
    pos = clust.centroid().x
    ax.plot(*pos, "x", markersize=7, color="orange", label="Cluster" if j==0 else "", alpha=0.8)
    ax.text(pos[0]+0.02, pos[1]+0.02, j)

# set limits
ax.set_xlim([-1, 2])
ax.set_ylim([-1, 1.8])
ax.set_aspect('equal', adjustable='box')
plt.legend()
plt.axis('off')
plt.tight_layout()
plt.savefig("figures/experiment_clusters.pdf")
plt.show()

In [ ]:
# set the prior parameters
params = [{"alpha": 0.5, "beta": 0.5} for _ in range(len(clusts))]

# check fov expectations for each cluster -- run for a few frames with noise
frames = 10
for _ in range(frames):
    for j_clust, clust in enumerate(clusts):
        psms = []
        for i_agent, agent in enumerate(agents):
            if np.linalg.norm(agent - clust.centroid().x) <= radius:
                # Get the PSM
                saw = i_agent in clust.agent_IDs  # did we see it?
                if saw:  # positive result
                    dets = clust.get_tracks_by_agent_ID(i_agent)
                    if len(dets) > 1:
                        raise RuntimeError
                    dist = clust.distance(dets[0])
                    p_exist = p_exist_fix if assign.has_assign(row=j_clust) else 0.5
                    p_exist = max(0.0, min(p_exist + (np.random.rand()-1/2)/4, 1.0))
                    psm = PSM(1.0, dist=dist, p_exist=p_exist)
                else:  # negative result
                    psm = PSM(0.0, p_exist=1.0)
                psms.append(psm)
            else:
                pass  # not expected to see
        
        # Each frame we add some uncertainty
        a = params[j_clust]["alpha"]
        b = params[j_clust]["beta"]
        m = a / (a + b)
        v = a + b
        m += (0.5 - m)/50
        v += 0.05
        params[j_clust]["alpha"] = m * v
        params[j_clust]["beta"]  = (1-m) * v

        # Update the parameters
        if len(psms) > 1:
            for psm in psms:
                params[j_clust]["alpha"] += psm.confidence * psm.value
                params[j_clust]["beta"]  += psm.confidence * (1 - psm.value)

# Plot results
from scipy.stats import beta

fig, ax = plt.subplots()

x = np.linspace(0, 1.0, 10000)
for j_clust, ps in enumerate(params):
    label = f"{j_clust}: True Object" if assign_truth.has_assign(row=j_clust) \
        else f"{j_clust}: False Pos."
    linestyle = "-" if "True" in label else "--"
    y = beta.pdf(x, ps["alpha"], ps["beta"])
    ax.plot(x, y, linewidth=2, linestyle=linestyle, label=label)
ax.set_xlim([0, 1.0])
ax.set_ylim([0, 10])
ax.legend(loc="upper left")
ax.set_xlabel("Trust Score")
plt.tight_layout()
plt.savefig("figures/experiment_trusts.pdf")
plt.show()